# A comparative evaluation on the generative ability of DCGANs

Kinga Jenei and Jonna Marie Matthiesen

Inspired by the pytorch DCGAN tutorial https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html.

In [ ]:
import random

import torch
from torch import optim
import torchvision.datasets as dset
import torchvision.transforms as transforms


from utils.figures import *
from utils.networks import *

In [ ]:
# Set random seed for reproducibility
manualSeed = '999'
random.seed(manualSeed)
torch.manual_seed(manualSeed)
print("Random Seed: ", manualSeed)

In [ ]:
dataroot = # dataroot must contain images inside folders
workers = 2 

# RGB images of shape 64x64
image_size = 64 

# model training
num_epochs = 30 
batch_size = 512 

# We experimented with different individual Learning rates for Generator and Discriminator
max_lr_D = 0.0002 
max_lr_G = 0.0002 
beta1 = 0.5 
ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print(f"Using device = {device}.")

In [ ]:
dataset = dset.ImageFolder(
    root=dataroot,
    transform=transforms.Compose(
        [
            transforms.Resize(image_size),
            transforms.CenterCrop(
                image_size
            ),  # Depending on the Dataset chose CenterCrop or RandomCrop
            transforms.RandomHorizontalFlip(0.5), # remove depending on the dataset
            transforms.RandomVerticalFlip(0.5), # remove depending on the dataset
            transforms.ToTensor(),  # image values in range [0, 1]
            transforms.Normalize(
                (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
            ),  # yields images with mean zero and values in range [-1, 1] - desired range since we are using tanh activation in the generator
        ]
    ),
)

dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, shuffle=True, num_workers=workers
)

print(f"Dataset size = {len(dataset)} images.")
imshow_grid(next(iter(dataloader)), device, "Training Images")

In [ ]:
netD, netG = create_dcgan(device)

# Model training 
criterion = nn.BCELoss()

# Create batch of latent vectors to visualize the progress of the generator
fixed_noise = torch.randn(64, 84, 1, 1, device=device)

real_label = 1
fake_label = 0

# Setup Adam optimizers for both Generator and Discriminator
optimizerD = optim.Adam(netD.parameters(), lr=max_lr_D, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=max_lr_G, betas=(beta1, 0.999))

In [ ]:
# Training Loop

# Keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")

for epoch in range(num_epochs):

    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        netD.zero_grad()

        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        output = netD(real_cpu).view(-1)
        out_size = output.size(0)

        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        noise = torch.randn(b_size, 84, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        #label = (0.1*torch.rand((b_size, 1))).to(device)

        output = netD(fake.detach()).view(-1)
        errD_fake =  criterion(output, label)

        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake

        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label) 
        output = netD(fake).view(-1)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

In [ ]:
# save models
torch.save(netD, "./netD.pb")
torch.save(netG, "./netG.pb")

In [ ]:
# plot results
losses(G_losses, D_losses)
real_vs_fakes(next(iter(dataloader)), device, img_list)
progress(img_list)